This notebook compares the scores of the local models and the model chain

In [1]:
import numpy as np
import pandas as pd
#import itertools
#import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error as mse, brier_score_loss
from chaining import Chain
import os

In [2]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [3]:
def missingness_and_categorical_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Stratify categorical variables
    for col in df.select_dtypes(include=['category']):
        counts = df[col].value_counts(normalize=True)
        for category in counts.index:
            idx = df[col] == category
            cv[idx] = cv[idx].fillna(np.random.choice(np.where(idx)[0], size=int(counts[category] * N_FOLDS), replace=False))

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [4]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

data = pd.read_csv(path)

# Bin the number of relapses into 0, 1, 2, 3 and 4+ 
def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)
data

USUBJID   AGE SEX       RACE      CONTINENT CESEV CECONTRT  TOTRELAP  \
0     MSOAC/0014  46.0   F        NaN            NaN   NaN      NaN       NaN   
1     MSOAC/0016   NaN   M      WHITE  NORTH AMERICA   NaN      NaN       NaN   
2     MSOAC/0019  44.0   M  NON-WHITE            NaN   NaN      NaN       NaN   
3     MSOAC/0024  60.0   M      WHITE  NORTH AMERICA   NaN      NaN       NaN   
4     MSOAC/0030  28.0   F      WHITE         EUROPE   NaN      NaN       NaN   
...          ...   ...  ..        ...            ...   ...      ...       ...   
2460  MSOAC/9986  46.0   M      WHITE        OCEANIA   NaN      NaN       NaN   
2461  MSOAC/9987  18.0   F        NaN            NaN   NaN      NaN       NaN   
2462  MSOAC/9995  38.0   F        NaN            NaN  MILD      NaN       4.0   
2463  MSOAC/9998  40.0   F      WHITE            NaN   NaN        Y       2.0   
2464  MSOAC/9999  38.0   F        NaN            NaN  MILD      NaN       2.0   

     MHCONTRT MHDIAGN  CARDIO  URINARY  MUSCKELET  FATIGUE  SMSTDY NRELAP  \
0         NaN    RRMS       0        0          0        0     NaN    0.0   
1           Y    SPMS       1        1          0        1     NaN    0.0   
2         NaN    PPMS       1        1          0        0     NaN    0.0   
3         NaN    SPMS       1        1          1        1     NaN    0.0   
4         NaN    RRMS       1        1          0        1     NaN    0.0   
...       ...     ...     ...      ...        ...      ...     ...    ...   
2460      NaN    RRMS       1        1          0        1     NaN    0.0   
2461      NaN    RRMS       0        0          0        0     NaN    0.0   
2462      NaN    RRMS       0        0          0        0   142.0    2.0   
2463        Y    PPMS       0        1          0        1    79.0    1.0   
2464      NaN    RRMS       0        0          0        0    69.0    2.0   

      NHPT-before  NHPT-2y  NHPT-after_2y  PASAT_2s-before  PASAT_2s-2y  \
0             NaN      NaN            NaN              NaN          NaN   
1             NaN      NaN            NaN              NaN          NaN   
2           23.65    21.30          20.15             34.5         35.5   
3           34.45    37.50            NaN             55.0         54.0   
4           16.55    17.90            NaN              NaN          NaN   
...           ...      ...            ...              ...          ...   
2460        19.35    18.95            NaN              NaN          NaN   
2461          NaN      NaN            NaN              NaN          NaN   
2462          NaN      NaN            NaN              NaN          NaN   
2463        23.80    22.40          22.50             21.5         30.5   
2464          NaN      NaN            NaN              NaN          NaN   

      PASAT_2s-after_2y  PASAT_3s-before  PASAT_3s-2y  PASAT_3s-after_2y  \
0                   NaN              NaN          NaN                NaN   
1                   NaN              NaN          NaN                NaN   
2                  43.0             43.5         51.0               53.0   
3                   NaN             60.0         60.0                NaN   
4                   NaN             58.0         60.0                NaN   
...                 ...              ...          ...                ...   
2460                NaN             58.0         60.0                NaN   
2461                NaN              NaN          NaN                NaN   
2462                NaN              NaN          NaN                NaN   
2463               33.5             31.5         39.5               40.5   
2464                NaN              NaN          NaN                NaN   

      SDMT-before  SDMT-2y  T25FW-before  T25FW-2y  T25FW-after_2y  T-before  \
0             NaN      NaN           NaN       NaN             NaN       NaN   
1             NaN      NaN          8.55      6.60             NaN       0.0   
2             NaN      NaN          6.30      6.15       

In [5]:
#variables = ['KFSS_M-2y', 'EDSS-2y', 'T25FW-2y', 'NRELAP']# removed KFSS_P-2y for now -- ('SMSTDY' gave a score of -0.03)
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'EDSS-after_2y', 'NRELAP', 'CESEV']

In [6]:
# Extract targets
targets = data[variables]

# Extract features by dropping the target columns
#features = data.drop(variables, axis=1)

columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE']
                    #'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'BDI-before', 'T-before','P-before','N-before']
# removed: 'NHPT-before', 'T25FW-before', 'SLEC_before','SES_before','EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before', 
# 'P_R36-SF12-before', 'R36-SF12-before_Ind',

features = data[columns_to_keep]
features

AGE SEX       RACE      CONTINENT MHDIAGN  CARDIO  URINARY  MUSCKELET  \
0     46.0   F        NaN            NaN    RRMS       0        0          0   
1      NaN   M      WHITE  NORTH AMERICA    SPMS       1        1          0   
2     44.0   M  NON-WHITE            NaN    PPMS       1        1          0   
3     60.0   M      WHITE  NORTH AMERICA    SPMS       1        1          1   
4     28.0   F      WHITE         EUROPE    RRMS       1        1          0   
...    ...  ..        ...            ...     ...     ...      ...        ...   
2460  46.0   M      WHITE        OCEANIA    RRMS       1        1          0   
2461  18.0   F        NaN            NaN    RRMS       0        0          0   
2462  38.0   F        NaN            NaN    RRMS       0        0          0   
2463  40.0   F      WHITE            NaN    PPMS       0        1          0   
2464  38.0   F        NaN            NaN    RRMS       0        0          0   

      FATIGUE  
0           0  
1           1  
2           0  
3           1  
4           1  
...       ...  
2460        1  
2461        0  
2462        0  
2463        1  
2464        0  

[2465 rows x 9 columns]

In [7]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
features.head()

AGE  CARDIO  URINARY  MUSCKELET  FATIGUE  SEX_F  SEX_M  RACE_NON-WHITE  \
0  46.0       0        0          0        0      1      0               0   
1   NaN       1        1          0        1      0      1               0   
2  44.0       1        1          0        0      0      1               1   
3  60.0       1        1          1        1      0      1               0   
4  28.0       1        1          0        1      1      0               0   

   RACE_WHITE  CONTINENT_ASIA  CONTINENT_EURASIA  CONTINENT_EUROPE  \
0           0               0                  0                 0   
1           1               0                  0                 0   
2           0               0                  0                 0   
3           1               0                  0                 0   
4           1               0                  0                 1   

   CONTINENT_NORTH AMERICA  CONTINENT_OCEANIA  CONTINENT_SOUTH AMERICA  \
0                        0                  0                        0   
1                        1                  0                        0   
2                        0                  0                        0   
3                        1                  0                        0   
4                        0                  0                        0   

   MHDIAGN_PPMS  MHDIAGN_RRMS  MHDIAGN_SPMS  
0             0             1             0  
1             0             0             1  
2             1             0             0  
3             0             0             1  
4             0             1             0

In [8]:
targets.dtypes

KFSS_M-2y           float64
KFSS_P-2y           float64
EDSS-2y             float64
T25FW-2y            float64
NHPT-2y             float64
P_R36-SF12-after    float64
M_R36-SF12-after    float64
SES_after           float64
SLEC_after          float64
KFSS_M-after_2y     float64
KFSS_P-after_2y     float64
EDSS-after_2y       float64
NRELAP               object
CESEV                object
dtype: object

In [9]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [10]:
# Generate CV folds
cv=missingness_and_categorical_stratified_cv(targets, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

features_cv['CV Fold'].value_counts()

CV Fold
4.0    510
3.0    502
0.0    500
1.0    495
2.0    458
Name: count, dtype: int64

Is it a problem that not all folds have the exact same number?

In [11]:
def normalized_mean_squared_error(true, pred, train):
    num = mse(true, pred)
    mean_value = np.mean(train)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

---

### Local Models

In [12]:
y_pred_list = []
y_test_list = []
y_train_list = []
y_pred_prob_list = []
yi_test_dummies_list = []
yi_train_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))
    y_train_list.append(pd.DataFrame(yi_train, columns=yi_train.columns, index=yi_train.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    subset_yi_train = yi_train.select_dtypes(include=['object'])
    yi_train_dummies = pd.get_dummies(subset_yi_train, columns=subset_yi_train.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate=False, #RUN LOCAL MODELS 
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    yi_train_dummies_list.append(yi_train_dummies)
    
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [13]:
yi_train_dummies_avg = []
i=0
# Calculate the percentage of 1s in each column
for yi_train_dummies_fold in yi_train_dummies_list:

    percentages = yi_train_dummies_fold.sum() / len(yi_train_dummies_fold)

    yi_train_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_list[i].index, columns=yi_train_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_train_dummies_avg_fold.columns:
        yi_train_dummies_avg_fold[col] = yi_train_dummies_avg_fold[col].apply(lambda x: percentages[col])
    
    i += 1
    yi_train_dummies_avg.append(yi_train_dummies_avg_fold)

In [14]:
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [15]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
#print("Normalized Brier scores for each level:")
#for variable_name, avg_score, std_score in scores_with_std:
#    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

In [16]:
# Initialize dictionary to store column sums
column_sums = {}

# Iterate over each dataframe in reorganized_dummy_list_first
for df in yi_test_dummies_list:
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Sum occurrences of 1s in the column and update column_sums
        column_sum = df[column].sum()
        column_sums[column] = column_sums.get(column, 0) + column_sum

# Create a dataframe from the column sums
total_counts_df = pd.DataFrame(list(column_sums.items()), columns=['Name', 'Total'])
#total_counts_df

In [17]:
# Calculate the weighted sum for NRELAP
nrelap_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("NRELAP"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        nrelap_weighted_sum += avg_score * count
        total_count += count
nrelap_weighted_avg = nrelap_weighted_sum / total_count

# Calculate the weighted sum for CESEV
cesev_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("CESEV"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        cesev_weighted_sum += avg_score * count
        total_count += count
cesev_weighted_avg = cesev_weighted_sum / total_count

# Print the results
print(f"Weighted avg for NRELAP: {nrelap_weighted_avg:.2f}")
print(f"Weighted avg for CESEV: {cesev_weighted_avg:.2f}")

cat_normalized_brier= [nrelap_weighted_avg, cesev_weighted_avg]

Weighted avg for NRELAP: 1.07
Weighted avg for CESEV: 1.96


In [18]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

#cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    #cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.15 (± 0.11)
CESEV: 1.80 (± 0.36)


In [19]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [20]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            y_train = y_train_list[fold_index][variable_name]

            score = normalized_mean_squared_error(y_test, y_pred, y_train)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]

# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (local):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (local):
KFSS_M-2y: 0.71 (± 0.07)
KFSS_P-2y: 0.87 (± 0.11)
EDSS-2y: 0.52 (± 0.03)
T25FW-2y: 1.22 (± 0.22)
NHPT-2y: 1.33 (± 0.33)
P_R36-SF12-after: 0.86 (± 0.02)
M_R36-SF12-after: 1.09 (± 0.03)
SES_after: 1.07 (± 0.10)
SLEC_after: 1.12 (± 0.07)
KFSS_M-after_2y: 0.75 (± 0.05)
KFSS_P-after_2y: 1.09 (± 0.22)
EDSS-after_2y: 0.60 (± 0.06)


In [21]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.7144871 , 0.86754384, 0.51699737, 1.22284979, 1.3320739 ,
       0.86166666, 1.08575278, 1.06705598, 1.12374131, 0.75011666,
       1.08720878, 0.60046788, 1.0726391 , 1.95922851])

In [22]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.07249766, 0.11373715, 0.02831994, 0.2236021 , 0.32955757,
       0.02470746, 0.0342752 , 0.10139115, 0.07338219, 0.05215299,
       0.2228481 , 0.06095894, 0.11462421, 0.36300708])

In [23]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 1.018702118992026


In [24]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.6293334323107214


### Propagate predictions

In [25]:
y_pred_list = []
y_test_list = []
y_train_list = []
y_pred_prob_list = []
yi_test_dummies_list = []
yi_train_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))
    y_train_list.append(pd.DataFrame(yi_train, columns=yi_train.columns, index=yi_train.index))


    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    subset_yi_train = yi_train.select_dtypes(include=['object'])
    yi_train_dummies = pd.get_dummies(subset_yi_train, columns=subset_yi_train.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred", #RUN MODELS IN A CHAIN
    )


    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    yi_train_dummies_list.append(yi_train_dummies)
    
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [26]:
yi_train_dummies_avg = []
i=0
# Calculate the percentage of 1s in each column
for yi_train_dummies_fold in yi_train_dummies_list:

    percentages = yi_train_dummies_fold.sum() / len(yi_train_dummies_fold)

    yi_train_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_list[i].index, columns=yi_train_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_train_dummies_avg_fold.columns:
        yi_train_dummies_avg_fold[col] = yi_train_dummies_avg_fold[col].apply(lambda x: percentages[col])
    
    i += 1
    yi_train_dummies_avg.append(yi_train_dummies_avg_fold)

In [27]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [28]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
#print("Normalized Brier scores for each level:")
#for variable_name, avg_score, std_score in scores_with_std:
#    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

In [29]:
# Initialize dictionary to store column sums
column_sums = {}

# Iterate over each dataframe in reorganized_dummy_list_first
for df in yi_test_dummies_list:
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Sum occurrences of 1s in the column and update column_sums
        column_sum = df[column].sum()
        column_sums[column] = column_sums.get(column, 0) + column_sum

# Create a dataframe from the column sums
total_counts_df = pd.DataFrame(list(column_sums.items()), columns=['Name', 'Total'])
#total_counts_df

In [30]:
# Calculate the weighted sum for NRELAP
nrelap_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("NRELAP"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        nrelap_weighted_sum += avg_score * count
        total_count += count
nrelap_weighted_avg = nrelap_weighted_sum / total_count

# Calculate the weighted sum for CESEV
cesev_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("CESEV"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        cesev_weighted_sum += avg_score * count
        total_count += count
cesev_weighted_avg = cesev_weighted_sum / total_count

# Print the results
print(f"Weighted avg for NRELAP: {nrelap_weighted_avg:.2f}")
print(f"Weighted avg for CESEV: {cesev_weighted_avg:.2f}")

cat_normalized_brier = [nrelap_weighted_avg, cesev_weighted_avg]

Weighted avg for NRELAP: 1.02
Weighted avg for CESEV: 1.78


In [31]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

#cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    #cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.09 (± 0.08)
CESEV: 1.66 (± 0.31)


In [32]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]


# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            y_train = y_train_list[fold_index][variable_name]

            score = normalized_mean_squared_error(y_test, y_pred, y_train)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - propagate predictions):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - propagate predictions):
KFSS_M-2y: 0.71 (± 0.07)
KFSS_P-2y: 0.82 (± 0.09)
EDSS-2y: 0.54 (± 0.04)
T25FW-2y: 1.16 (± 0.12)
NHPT-2y: 1.16 (± 0.27)
P_R36-SF12-after: 0.90 (± 0.04)
M_R36-SF12-after: 1.15 (± 0.08)
SES_after: 1.07 (± 0.09)
SLEC_after: 1.17 (± 0.10)
KFSS_M-after_2y: 0.80 (± 0.03)
KFSS_P-after_2y: 1.14 (± 0.15)
EDSS-after_2y: 0.68 (± 0.08)


In [33]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.7144871 , 0.82008945, 0.53679771, 1.15815499, 1.163118  ,
       0.8959963 , 1.1547296 , 1.06726975, 1.1652273 , 0.79620393,
       1.13779282, 0.68285029, 1.02286867, 1.78158394])

In [34]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.07249766, 0.09334165, 0.03613727, 0.11660765, 0.27366932,
       0.04361651, 0.07540907, 0.08562591, 0.09624444, 0.03210696,
       0.15375922, 0.07552454, 0.07816503, 0.30706341])

In [35]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 1.0069407032933622


In [36]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.5086500270994773


### Propagate true values

In [41]:
y_pred_list = []
y_test_list = []
y_train_list = []
y_pred_prob_list = []
yi_test_dummies_list = []
yi_train_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))
    y_train_list.append(pd.DataFrame(yi_train, columns=yi_train.columns, index=yi_train.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    subset_yi_train = yi_train.select_dtypes(include=['object'])
    yi_train_dummies = pd.get_dummies(subset_yi_train, columns=subset_yi_train.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="true", #RUN MODELS IN A CHAIN
    )


    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    yi_train_dummies_list.append(yi_train_dummies)
    
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [42]:
yi_train_dummies_avg = []
i=0
# Calculate the percentage of 1s in each column
for yi_train_dummies_fold in yi_train_dummies_list:

    percentages = yi_train_dummies_fold.sum() / len(yi_train_dummies_fold)

    yi_train_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_list[i].index, columns=yi_train_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_train_dummies_avg_fold.columns:
        yi_train_dummies_avg_fold[col] = yi_train_dummies_avg_fold[col].apply(lambda x: percentages[col])
    
    i += 1
    yi_train_dummies_avg.append(yi_train_dummies_avg_fold)

In [43]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [44]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
#print("Normalized Brier scores for each level:")
#for variable_name, avg_score, std_score in scores_with_std:
#    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

In [45]:
# Initialize dictionary to store column sums
column_sums = {}

# Iterate over each dataframe in reorganized_dummy_list_first
for df in yi_test_dummies_list:
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Sum occurrences of 1s in the column and update column_sums
        column_sum = df[column].sum()
        column_sums[column] = column_sums.get(column, 0) + column_sum

# Create a dataframe from the column sums
total_counts_df = pd.DataFrame(list(column_sums.items()), columns=['Name', 'Total'])
#total_counts_df

In [46]:
# Calculate the weighted sum for NRELAP
nrelap_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("NRELAP"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        nrelap_weighted_sum += avg_score * count
        total_count += count
nrelap_weighted_avg = nrelap_weighted_sum / total_count

# Calculate the weighted sum for CESEV
cesev_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("CESEV"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        cesev_weighted_sum += avg_score * count
        total_count += count
cesev_weighted_avg = cesev_weighted_sum / total_count

# Print the results
print(f"Weighted avg for NRELAP: {nrelap_weighted_avg:.2f}")
print(f"Weighted avg for CESEV: {cesev_weighted_avg:.2f}")

cat_normalized_brier= [nrelap_weighted_avg, cesev_weighted_avg]

Weighted avg for NRELAP: 1.01
Weighted avg for CESEV: 1.77


In [47]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_train_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

#cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    #cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.03 (± 0.05)
CESEV: 1.54 (± 0.51)


In [48]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [49]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            y_train = y_train_list[fold_index][variable_name]

            score = normalized_mean_squared_error(y_test, y_pred, y_train)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.71 (± 0.07)
KFSS_P-2y: 0.81 (± 0.08)
EDSS-2y: 0.50 (± 0.02)
T25FW-2y: 0.99 (± 0.07)
NHPT-2y: 0.93 (± 0.05)
P_R36-SF12-after: 0.83 (± 0.03)
M_R36-SF12-after: 1.02 (± 0.05)
SES_after: 0.99 (± 0.04)
SLEC_after: 1.05 (± 0.06)
KFSS_M-after_2y: 0.71 (± 0.03)
KFSS_P-after_2y: 1.00 (± 0.14)
EDSS-after_2y: 0.57 (± 0.04)


In [50]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.7144871 , 0.80575847, 0.49564878, 0.99310597, 0.93164642,
       0.83115637, 1.01895165, 0.99268779, 1.04569312, 0.71179869,
       1.00387099, 0.57478256, 1.00778209, 1.77476551])

In [51]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.07249766, 0.08231472, 0.02248682, 0.06899315, 0.05191767,
       0.03324506, 0.04950392, 0.04363207, 0.06240683, 0.03327623,
       0.14057758, 0.04182778, 0.05240342, 0.51275679])

In [52]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.921581106947109


In [53]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.5635881449558359
